In [1]:
import numpy as np
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)


#bring in dipoles and energies
dipoles = np.load("LiH_r_1.4_6311g_fci_Dipoles.npy")
energies = np.load("LiH_r_1.4_6311g_fci_Energies.npy")
#print(energies.shape)
print(dipoles)



#lambda vectors 
lambda_vec = np.array([0.0, 0.0, 0.05])




omega = 0.12086 


m_n = 0 #num_photons


mu_n = 0 #electronic state


n_elec = 50



[[[-6.58046359e-15  3.41157881e-14 -2.09867294e+00]
  [ 2.78190247e-15 -3.60685480e-14  5.86355257e-13]
  [-4.30257733e-15  4.18478564e-14 -9.00621585e-01]
  [ 3.67415785e-14 -1.19765363e-13 -6.68352408e-14]
  [-4.96295546e-13 -7.85195887e-14  1.49124188e-15]
  [-2.66638853e-01  1.45745802e+00  6.76181589e-14]
  [ 1.45745802e+00  2.66638853e-01  1.85958696e-15]
  [-2.19653815e-15  1.01654789e-14 -3.22935736e-13]
  [-4.78701557e-15 -4.32034492e-14  5.36954371e-01]
  [-1.33057272e-14  6.51292020e-15  7.13860627e-14]
  [ 9.14837705e-16 -1.72261290e-14  1.11070765e-01]
  [-2.44543196e-13  1.54376129e-13 -8.98859661e-15]
  [-2.11504775e-13  1.33374181e-13  1.45109293e-14]
  [-2.07803224e-01  2.19737473e-01 -4.26851643e-15]
  [ 2.19737473e-01  2.07803224e-01  8.81711317e-15]
  [ 1.68625853e-15 -4.38057258e-16  1.01528882e-13]
  [-3.64242388e-15  8.90882132e-15  8.95097334e-01]
  [ 4.89698791e-15 -6.46731251e-16  5.88933600e-14]
  [-1.51837150e-15  6.97479086e-15 -1.68643383e-01]
  [ 4.603352

In [4]:

def build_d_matrix(lambda_vec, dipoles):
    #return np.einsum('k, ijk -> ij', lambda_vec, dipoles[:mu_n, : mu_n, :])
    return np.einsum('k, ijk -> ij', lambda_vec, dipoles)
    return np.einsum('k, ijk -> ij', lambda_vec, dipoles[:n_elec, : n_elec, :])
    #return np.tensordot(dipoles, lambda_vec, axes = 1)

def build_d_matrix_2(lambda_vec, dipoles, n_el):
        _d = np.zeros((n_el, n_el))
        for a in range(n_el):
            for b in range(n_el):
                _d[a, b] = np.dot(lambda_vec, dipoles[a, b, :])

        return _d




d_matrix_1 = build_d_matrix(lambda_vec, dipoles)
d_matrix_2 = build_d_matrix_2(lambda_vec, dipoles, n_elec)

print(np.shape(dipoles))

for i in range(4):
    for j in range(4):
        
        print(i,j,dipoles[i,j,:])
        d_ij = np.dot(lambda_vec, dipoles[i,j,:])
        print(d_ij)
        assert np.isclose(d_matrix_1[i,j], d_ij)


assert np.allclose(d_matrix_1, d_matrix_2)

(50, 50, 3)
0 0 [-6.58046359e-15  3.41157881e-14 -2.09867294e+00]
-0.10493364689983152
0 1 [ 2.78190247e-15 -3.60685480e-14  5.86355257e-13]
2.9317762858220245e-14
0 2 [-4.30257733e-15  4.18478564e-14 -9.00621585e-01]
-0.04503107925280421
0 3 [ 3.67415785e-14 -1.19765363e-13 -6.68352408e-14]
-3.3417620406696223e-15
1 0 [ 2.78190247e-15 -3.60685480e-14  5.86355257e-13]
2.931776285822026e-14
1 1 [ 6.81405853e-15 -5.39611609e-14  1.75871551e+00]
0.0879357755407531
1 2 [-2.19483815e-14 -7.57900932e-14 -9.36131178e-13]
-4.68065588840095e-14
1 3 [3.65409022e-01 2.17874898e+00 2.08800640e-14]
1.0440032022605904e-15
2 0 [-4.30257733e-15  4.18478564e-14 -9.00621585e-01]
-0.04503107925280418
2 1 [-2.19483815e-14 -7.57900932e-14 -9.36131178e-13]
-4.6806558884009464e-14
2 2 [ 1.61915251e-14 -1.48046613e-13  2.08193768e+00]
0.10409688396788078
2 3 [-4.32634119e-14  1.45559751e-13  1.61253535e-13]
8.062676737884662e-15
3 0 [ 3.67415785e-14 -1.19765363e-13 -6.68352408e-14]
-3.341762040669621e-15
3 1 

In [5]:
d_matrix = np.copy(d_matrix_1)
#d_matrix = d_matrix + d_matrix.T + np.diag(np.diag(d_matrix))
#d_matrix = d_matrix + d_matrix.T
print(d_matrix[0:4, 0:4])



def calc_first_order_energy_correction(d_matrix, omega, mu_n):

    E_n_1 = 0
    for gamma in range(0, len(energies)):
        E_n_1 += d_matrix[mu_n][gamma]*d_matrix[gamma][mu_n]

    E_n_1 = omega* E_n_1

    return E_n_1                




def delta(i,j):
    if i ==j:
        return 1
    else:
        return 0

#unsimplified version of second order energy correction to check if the same answer is gotten
def calc_second_order_energy_correction_unsimplified(d_matrix, omega, mu_n, energies, m_n):
    E_n_2 = 0


    for mu_l in range(0, len(energies)):

            #for m_l in range(max(m_n-1,0), m_n+2):
            for m_l in range(max(0, m_n-1), m_n+2 ):

                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    sum_over_energy_states_gamma = 0
                    for gamma in range(0, len(energies)):
                        #numerator += d_matrix[gamma][mu_l]*d_matrix[mu_l][gamma]
                        sum_over_energy_states_gamma += (d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n] * delta(m_l, m_n))

                    
                    E_n_2 += (((-omega * d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n + 1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (omega * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega))

                    #print((((-omega * d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n +1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (omega * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega)))


    return E_n_2




def calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n):
    E_n_2 = 0


    for mu_l in range(0, len(energies)):
            for m_l in range(max(m_n-1,0), m_n+2):
                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    if m_l == m_n + 1:
                        E_n_2 +=  ( (d_matrix[mu_l][mu_n] * np.sqrt(m_n+1)) ** 2 )/(energies[mu_n] - energies[mu_l] - omega)

                        #print( d_matrix[mu_l][mu_n]  )


                    elif m_l  == m_n - 1:
                        E_n_2 +=  ( (d_matrix[mu_l][mu_n] * np.sqrt(m_n)) ** 2 )/(energies[mu_n] - energies[mu_l] + omega)

                        #print(( (d_matrix[mu_l][mu_n] * np.sqrt(m_n+1)) ** 2 )/(energies[mu_n] - energies[mu_l] - omega))

        
                    elif m_l  == m_n:

                        
                        numerator = 0
                        for gamma in range(0, len(energies)):
                            numerator += d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n]

                            #print("numerator: ", numerator)


                            #print("d_matrix[mu_l][gamma]: ",  d_matrix[mu_l][gamma] ," d_matrix[gamma][mu_n]: ", d_matrix[gamma][mu_n])
                

                        E_n_2 += (numerator**2) / ((energies[mu_n] - energies[mu_l]))

                        #print("denominator:" , ((energies[mu_n] - energies[mu_l])))

                        #print("contrib: ",(numerator**2) / ((energies[mu_n] - energies[mu_l])) )



    E_n_2 = E_n_2 * omega**2
    return E_n_2




def calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2):
    return energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 

def calc_energy_correction_to_third_order(mu_n, E_n_1, E_n_2, E_n_3):
    return energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 + E_n_3



E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)


E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)



print("E_n_1: ", E_n_1)
print("E-n_2: ", E_n_2)


print("uncorrected_energy:" , energies[mu_n]+(omega * m_n))

print("energy corrected to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))

qed_fci_energies = np.load("LiH_r_1.4_6311g_qedfci_Energies.npy")
print("qed_fci_energy:", qed_fci_energies[mu_n])

[[-1.04933647e-01  2.93177629e-14 -4.50310793e-02 -3.34176204e-15]
 [ 2.93177629e-14  8.79357755e-02 -4.68065589e-14  1.04400320e-15]
 [-4.50310793e-02 -4.68065589e-14  1.04096884e-01  8.06267674e-15]
 [-3.34176204e-15  1.04400320e-15  8.06267674e-15  4.16640161e-03]]
E_n_1:  0.002139800021418779
E-n_2:  -0.0016045019835372299
uncorrected_energy: -8.012194758186697
energy corrected to second order:  -8.011659460148815
qed_fci_energy: -8.00913664849758
